In [89]:
import time
def generate_accuracy_and_heatmap(model, x, y):
#     cm = confusion_matrix(y,model.predict(x))
#     sns.heatmap(cm,annot=True,fmt="d")

    
    ac = accuracy_score(y,model.predict(x))
    f_score = f1_score(y,model.predict(x))
    
    #ac = accuracy_score(y,model.predict(x))
    #f_score = f1_score(y,model.predict(x))
    print('Accuracy is: ', ac)
    print('F1 score is: ', f_score)
    #print ("\n")
    #print (pd.crosstab(pd.Series(model.predict(x), name='Predicted'),
                     #  pd.Series(y['Outcome'],name='Actual')))
    return 1

# importing the libraries

In [90]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

data = pd.read_csv("dataset.csv") #importing dataset
data_new=data.copy()
y=data.iloc[18500:20500 , -1]
x=data.iloc[18500:20500 , 0:30]
data_new.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [91]:
# checking for missing values

x.info()
y.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 18500 to 20499
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0    Destination Port             2000 non-null   int64  
 1    Flow Duration                2000 non-null   int64  
 2    Total Fwd Packets            2000 non-null   int64  
 3    Total Backward Packets       2000 non-null   int64  
 4   Total Length of Fwd Packets   2000 non-null   int64  
 5    Total Length of Bwd Packets  2000 non-null   int64  
 6    Fwd Packet Length Max        2000 non-null   int64  
 7    Fwd Packet Length Min        2000 non-null   int64  
 8    Fwd Packet Length Mean       2000 non-null   float64
 9    Fwd Packet Length Std        2000 non-null   float64
 10  Bwd Packet Length Max         2000 non-null   int64  
 11   Bwd Packet Length Min        2000 non-null   int64  
 12   Bwd Packet Length Mean       2000 non-null   float64
 13

DDoS      1617
BENIGN     383
Name:  Label, dtype: int64

In [92]:
# transformation of categorical features
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y=label_encoder.fit_transform(y)
print(y)

[0 0 0 ... 1 1 1]


In [93]:
# splitting dataset to train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,  random_state=0)

# model building 

In [94]:
from sklearn.linear_model import LogisticRegression
# starting time
start = time.time()
clf_lr = LogisticRegression() 
lr_baseline_model=clf_lr.fit(x_train,y_train)
Y_lr_predict=clf_lr.predict(x_test)
y_pred=clf_lr.predict(x_test)
# End time
end = time.time()
print(end-start)

0.21968960762023926


C:\Users\Subinoy Mukherjee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [95]:
generate_accuracy_and_heatmap(lr_baseline_model, x_test, y_test)

Accuracy is:  0.984
F1 score is:  0.9900249376558605


1

In [96]:
confusion_matrix(y_test,y_pred)

array([[ 95,   5],
       [  3, 397]], dtype=int64)

# Feature selection

In [97]:
# applying SelectKBest features

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=17)
fit = bestfeatures.fit(x_train,y_train)
print(fit)

SelectKBest(k=17, score_func=<function chi2 at 0x00000206A31EA318>)


In [98]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

#concat two dataframes for better visualization 

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
featureScores

,Specs,Score
0,Destination Port,4.245421e+06
1,Flow Duration,5.212250e+08
2,Total Fwd Packets,3.846463e+00
3,Total Backward Packets,2.670125e+01
4,Total Length of Fwd Packets,5.081671e+05
5,Total Length of Bwd Packets,1.572439e+06
6,Fwd Packet Length Max,1.266325e+05
7,Fwd Packet Length Min,2.641989e+04
8,Fwd Packet Length Mean,5.440038e+04
9,Fwd Packet Length Std,5.007580e+04


In [99]:
print(featureScores.nlargest(17,'Score'))  #print 17 best features

                Specs         Score
14       Flow Bytes/s  5.661222e+09
25      Bwd IAT Total  1.215321e+09
29        Bwd IAT Min  1.140513e+09
24        Fwd IAT Min  1.079677e+09
18       Flow IAT Max  9.007761e+08
23        Fwd IAT Max  8.631281e+08
26       Bwd IAT Mean  7.533264e+08
22        Fwd IAT Std  6.338895e+08
1       Flow Duration  5.212250e+08
20      Fwd IAT Total  5.088833e+08
17       Flow IAT Std  3.386664e+08
28        Bwd IAT Max  3.325449e+08
16      Flow IAT Mean  1.334543e+08
15     Flow Packets/s  5.676441e+07
21       Fwd IAT Mean  1.153891e+07
27        Bwd IAT Std  7.928623e+06
0    Destination Port  4.245421e+06


In [100]:
x_train_chi=bestfeatures.transform(x_train)
x_test_chi=bestfeatures.transform(x_test)

In [101]:
x_train.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min
19545,80,722684,4,0,24,0,6,6,6.000000,0.000000,...,722684,240894.6667,416522.147400,721853,313,0,0.000000,0.00000,0,0
19437,80,705595,3,5,26,11607,20,0,8.666667,10.263203,...,715,357.5000,444.770165,672,43,705546,176386.500000,352169.12900,704640,4
20158,80,730299,4,0,24,0,6,6,6.000000,0.000000,...,730299,243433.0000,420542.954100,729034,275,0,0.000000,0.00000,0,0
19029,80,21958,3,4,26,11601,20,0,8.666667,10.263203,...,19922,9961.0000,13867.778190,19767,155,21896,7298.666667,10960.50137,19933,339
19395,80,373815,3,6,26,11601,20,0,8.666667,10.263203,...,11122,5561.0000,7796.559369,11074,48,373791,74758.200000,160763.26100,362214,49


In [102]:
x_train_chi[0:2]

array([[8.00000000e+01, 7.22684000e+05, 3.32095356e+01, 5.53492259e+00,
        2.40894667e+05, 4.16522147e+05, 7.21853000e+05, 7.22684000e+05,
        2.40894667e+05, 4.16522147e+05, 7.21853000e+05, 3.13000000e+02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [8.00000000e+01, 7.05595000e+05, 1.64867948e+04, 1.13379488e+01,
        1.00799286e+05, 2.66268814e+05, 7.04640000e+05, 7.15000000e+02,
        3.57500000e+02, 4.44770165e+02, 6.72000000e+02, 4.30000000e+01,
        7.05546000e+05, 1.76386500e+05, 3.52169129e+05, 7.04640000e+05,
        4.00000000e+00]])

# model building after Feature Selection
# 1)Logistic Regression


In [103]:
from sklearn.linear_model import LogisticRegression
start = time.time()
clf_lr = LogisticRegression() 
lr_baseline_model=clf_lr.fit(x_train_chi,y_train)
y_pred=clf_lr.predict(x_test_chi)
end = time.time()
print(end-start)

0.16493654251098633


C:\Users\Subinoy Mukherjee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [104]:
generate_accuracy_and_heatmap(lr_baseline_model, x_test_chi, y_test)

Accuracy is:  0.976
F1 score is:  0.9849624060150376


1

In [105]:
confusion_matrix(y_test,y_pred)

array([[ 95,   5],
       [  7, 393]], dtype=int64)

In [106]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       100
           1       0.99      0.98      0.98       400

    accuracy                           0.98       500
   macro avg       0.96      0.97      0.96       500
weighted avg       0.98      0.98      0.98       500



# 2) KNN 

In [107]:
from sklearn.neighbors import KNeighborsClassifier
start = time.time()
clf = KNeighborsClassifier()
baseline_model=clf.fit(x_train_chi,y_train)
y_pred=clf.predict(x_test_chi)
end = time.time()
print(end-start)

0.06486821174621582


In [108]:
generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

Accuracy is:  0.98
F1 score is:  0.9874055415617129


1

In [109]:
confusion_matrix(y_test,y_pred)

array([[ 98,   2],
       [  8, 392]], dtype=int64)

In [110]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       100
           1       0.99      0.98      0.99       400

    accuracy                           0.98       500
   macro avg       0.96      0.98      0.97       500
weighted avg       0.98      0.98      0.98       500



# 3) SVM

In [111]:
'''
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_train_chi, y_train)
y_pred = classifier.predict(x_test_chi)
'''

"\nfrom sklearn.svm import SVC\nclassifier = SVC(kernel = 'linear', random_state = 0)\nclassifier.fit(x_train_chi, y_train)\ny_pred = classifier.predict(x_test_chi)\n"

In [112]:
#generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

In [113]:
#confusion_matrix(y_test,y_pred)

In [114]:
#print(classification_report(y_test,y_pred))

# 4) Descision tree

In [115]:
from sklearn.tree import DecisionTreeClassifier
start = time.time()
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(x_train_chi, y_train)
y_pred = classifier.predict(x_test_chi)
end = time.time()
print(end-start)

0.030181169509887695


In [116]:
generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

Accuracy is:  0.98
F1 score is:  0.9874055415617129


1

In [117]:
confusion_matrix(y_test,y_pred)

array([[ 97,   3],
       [  0, 400]], dtype=int64)

In [118]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       100
           1       0.99      1.00      1.00       400

    accuracy                           0.99       500
   macro avg       1.00      0.98      0.99       500
weighted avg       0.99      0.99      0.99       500



# 5)Random Forest

In [119]:
from sklearn.ensemble import RandomForestClassifier
start = time.time()
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(x_train_chi, y_train)
y_pred = classifier.predict(x_test_chi)
end = time.time()
print(end-start)

0.0699467658996582


In [120]:
generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

Accuracy is:  0.98
F1 score is:  0.9874055415617129


1

In [121]:
confusion_matrix(y_test,y_pred)

array([[ 97,   3],
       [  1, 399]], dtype=int64)

In [122]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       100
           1       0.99      1.00      1.00       400

    accuracy                           0.99       500
   macro avg       0.99      0.98      0.99       500
weighted avg       0.99      0.99      0.99       500



# 6)Naive Byes

In [123]:
from sklearn.naive_bayes import GaussianNB
start = time.time()
classifier = GaussianNB()
classifier.fit(x_train_chi, y_train)
y_pred = classifier.predict(x_test_chi)
end = time.time()
print(end-start)

0.0045239925384521484


In [124]:
generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

Accuracy is:  0.98
F1 score is:  0.9874055415617129


1

In [125]:
confusion_matrix(y_test,y_pred)

array([[ 11,  89],
       [  2, 398]], dtype=int64)

In [126]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.11      0.19       100
           1       0.82      0.99      0.90       400

    accuracy                           0.82       500
   macro avg       0.83      0.55      0.55       500
weighted avg       0.82      0.82      0.76       500



# 7) Kernel SVM

In [131]:
from sklearn.svm import SVC
start = time.time()
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(x_train_chi, y_train)
y_pred = classifier.predict(x_test_chi)
end = time.time()
print(end-start)

0.06017160415649414


In [132]:
generate_accuracy_and_heatmap(baseline_model, x_test_chi, y_test)

Accuracy is:  0.98
F1 score is:  0.9874055415617129


1

In [133]:
confusion_matrix(y_test,y_pred)

array([[  3,  97],
       [  0, 400]], dtype=int64)

In [134]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.03      0.06       100
           1       0.80      1.00      0.89       400

    accuracy                           0.81       500
   macro avg       0.90      0.52      0.48       500
weighted avg       0.84      0.81      0.73       500

